# Clasificador de Noticias

## Tarea Metodos No Supervisados
Nombre: Jaime Toribio

In [97]:
from urllib.request import FancyURLopener
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# 1. Carga de datos

Utilizare scrapping para obtener las noticias con las que vamos a entrenar el modelo. Podria barrer la web de El Comercio pero esta web se actualiza constantenmente. Para obtener un resultado reproducible utilizare una lista de links cuyo contenido servira para entrenar el modelo de clasificación.

In [98]:
class MyOpener(FancyURLopener):
    version = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11'
myopener = MyOpener()

/Users/Jaime/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
links = pd.read_csv('./Data/links noticias.txt')

In [100]:
links.head()

,links
0,https://elcomercio.pe/deporte-total/seleccion/...
1,https://elcomercio.pe/deporte-total/champions-...
2,https://elcomercio.pe/deporte-total/futbol-per...
3,https://elcomercio.pe/tecnologia/actualidad/fu...
4,https://elcomercio.pe/deporte-total/futbol-mun...


In [101]:
#validando la carga
links['links'][0]

'https://elcomercio.pe/deporte-total/seleccion/paolo-guerrero-brasil-confian-juegue-final-copa-sudamericana-noticia-479074'

In [102]:
#Cantidad de links
len(links)

28

## Analizando la estructura del HTML:

El contenido real de la noticia se encuentra en los siguientes elementos HTML (objeto-clase):

1. Titulo: h1 news-title
2. Resumen: h2 news-summary
3. Descripción de la foto: p foto-description
4. Contenido: p parrafo first-parrafo

NOTA:El en caso del comercio antes de empezar el contenido de la noticia esta el nombre del autor, que en realidad viene a ser como una categoria. Por ejem: Deporte Total para deportes. No lo incluiré ya que podria sobreentrenarse el modelo.

A continuación ejemplo para 1 link:

In [103]:
#conexion
URL='https://elcomercio.pe/deporte-total/seleccion/paolo-guerrero-brasil-confian-juegue-final-copa-sudamericana-noticia-479074'
html=myopener.open(URL).read()
soup=BeautifulSoup(html,"lxml")

print('='*100)

In [104]:
titulo=soup.find_all('h1',class_='news-title')
titulo[0].get_text()

'Paolo Guerrero: confían en que juegue final de Sudamericana'

In [105]:
resumen=soup.find_all('h2',class_='news-summary')
resumen[0].get_text()


'A pesar de que la FIFA decidió extender por 10 días más la suspensión a Paolo Guerrero, en Brasil esperan que pueda estar con Flamengo en el partido de vuelta de la final de la Copa Sudamericana'

In [106]:
foto_desc=soup.find_all('p',class_='foto-description')
foto_desc[0].get_text()

'En Brasil esperan que Paolo Guerrero pueda llegar a jugar alguno de los dos partidos de la final de la Copa Sudamericana entre Flamengo e Independiente. (Foto: AP)'

In [107]:
lista=soup.find_all('p',class_='parrafo first-parrafo')+soup.find_all('p',class_='parrafo first-parrafo ')
contenido = ''
for i in range(len(lista)):
    contenido = contenido + lista[i].get_text()
contenido

'En Brasil todavía tienen esperanza de ver a Paolo Guerrero jugar con Flamengo antes de que acabe el año. A pesar de que la FIFA extendió por 10 días más la suspensión provisional al delantero de la selección peruana, el periodista de "O\'Globo", Diogo Dantas, cree que es posible que el jugador llegue al segundo duelo de la final de la Copa Sudamericana contra Independiente de Avellaneda."Hay optimismo en Flamengo porque hay una final de Copa Sudamericana el próximo miércoles y todos cuentan con el retorno de Paolo Guerrero. Toda la situación en FIFA, sus abogados salieron muy optimistas. El hecho de no tener problemas con cocaína, el examen capilar, las pruebas son muy consistentes. Hay una posibilidad muy grande que la respuesta sea muy positiva", afirmó el periodista brasileño a "RPP".Y agregó: "Si un castigo ocurre, el máximo previsto aquí es de 60 días. Ya se cumplieron 30 días de sanción provisoria, volvería en 2018 para jugar la Copa Libertadores con Flamengo".Como se recuerda, 

## Guardando el contenido

Vamos a barrer todos los links del archivo y guardarlos en distintos archivos. Uno por cada noticia. Para el entrenamiento del modelo concatenaré todos los archivos. El objetivo de tenerlos en distintos archivos es para luego poder hacer un vector promedio por cada archivo.

In [108]:
noticias = []
for i in range(len(links)):
    print(links['links'][i])
    #abre conexion con la web
    URL=links['links'][i]
    html=myopener.open(URL).read()
    soup=BeautifulSoup(html,"lxml")

    #busca contenido de la noticia
    t=soup.find_all('h1',class_='news-title')
    r=soup.find_all('h2',class_='news-summary')
    f=soup.find_all('p',class_='foto-description')
    p=soup.find_all('p',class_='parrafo first-parrafo')+soup.find_all('p',class_='parrafo first-parrafo ')
    
    #algunas noticias no tienen resumen o descripcion de foto, por ello si no existe asignamos '' a la variable.
    titulo = t[0].get_text() if len(t)>0 else ''
    resumen = r[0].get_text() if len(r)>0 else ''
    foto_desc = f[0].get_text() if len(f)>0 else ''
    
    contenido = ''
    for i in range(len(p)):
        contenido = contenido + p[i].get_text()
    
    #genera la noticia
    noticia = [titulo,
               resumen,
               foto_desc,
               contenido]
    
    #agrega a la lista de noticias
    noticias.append(noticia)
    

df = pd.DataFrame(noticias, columns = ['titulo', 'resumen', 'foto_desc', 'contenido'])

#guarda en un archivo delimitado por pipes |
#np.savetxt('noticias.txt', df, fmt='%s', delimiter='|', header='titulo|resumen|foto_desc|contenido')
    
#Muchos problemas con el encoding, por mientras trabajare en memoria.
#tuve que usar enconding para poder guardar pero luego para leer era complicado: titulo.encode('utf-8') 
    

https://elcomercio.pe/deporte-total/seleccion/paolo-guerrero-brasil-confian-juegue-final-copa-sudamericana-noticia-479074
https://elcomercio.pe/deporte-total/champions-league/juventus-vs-olympiacos-vivo-online-grupo-d-champions-league-noticia-479058
https://elcomercio.pe/deporte-total/futbol-peruano/alianza-lima-gustavo-zevallos-juntaremos-semana-pablo-bengoechea-noticia-479001
https://elcomercio.pe/tecnologia/actualidad/futbol-peruano-club-mejor-hinchada-digital-noticia-479026
https://elcomercio.pe/deporte-total/futbol-mundial/barcelona-vs-sporting-lisboa-vivo-online-champions-league-noticia-478842
https://elcomercio.pe/deporte-total/futbol-peruano/alianza-lima-vs-real-garcilaso-frases-polemicas-fuera-canchas-noticia-478911
https://elcomercio.pe/deporte-total/futbol-mundial/carlo-ancelotti-interesado-dirigir-italia-noticia-478943
https://elcomercio.pe/deporte-total/polideportivo/wwe-monday-night-vivo-online-angeles-roman-reigns-vengara-samoa-joe-noticia-478782
https://elcomercio.pe/de

In [109]:
df.head()

,titulo,resumen,foto_desc,contenido
0,Paolo Guerrero: confían en que juegue final de...,A pesar de que la FIFA decidió extender por 10...,En Brasil esperan que Paolo Guerrero pueda lle...,En Brasil todavía tienen esperanza de ver a Pa...
1,Juventus ganó 2-0 a Olympiacos y pasó a octavo...,\nJuventus logró su boleto a la siguiente inst...,Juventus logró su boleto a la siguiente instan...,La Juventus se clasificó para los octavos de f...
2,"Alianza Lima: ""Nos juntaremos esta semana con ...","El gerente deportivo de Alianza Lima, Gustavo ...",En Alianza Lima confían en retener a Pablo Ben...,"Gustavo Zevallos, gerente deportivo de Alianza..."
3,¿Qué equipo peruano tiene la mejor hinchada en...,La interacción en redes social como Facebook y...,Los clubes peruanos dan sus últimas noticias m...,"Facebook, Instagram y Twitter se han vuelto un..."
4,Barcelona ganó 2-0 a Sporting de Lisboa por Ch...,"\nBarcelona, que alineó varios suplentes y dej...",Barcelona ganó al Sporting de Lisboa con un go...,Barcelona se deshizo sin problemas (2-0) de un...


In [110]:
#noticias_load = pd.read_csv('noticias.txt', delimiter = '|', encoding='utf-8')
#noticias_load

# 2. Entrenando el modelo word2vec

In [111]:
from gensim.models import word2vec

In [112]:
#sentences = word2vec.Text8Corpus('./Data/links noticias.txt')

In [124]:
dimensiones = 4
modelo = word2vec.Word2Vec([s.lower().split() for s in df['contenido']],size=dimensiones,min_count=3,)

In [125]:
for i in modelo.wv.vocab:
    print(i)

en
todavía
tienen
de
ver
a
paolo
jugar
con
flamengo
antes
que
el
la
por
10
días
más
al
es
posible
segundo
final
copa
sudamericana
contra
independiente
porque
hay
una
próximo
y
todos
toda
sus
muy
hecho
no
tener
problemas
las
pruebas
son
respuesta
sea
afirmó
brasileño
"si
un
aquí
ya
se
2018
para
partido
entre
e
este
mientras
podría
si
juventus
los
octavos
champions
league
tras
olympiacos
del
grupo
sporting
goles
fueron
juan
su
lo
cual
le
portugal
esa
equipo
italiano
fue
primer
disparo
centro
desde
área
gol
aún
vio
rechazó
líneas
volvió
mano
frente
empate
real
última
esta
temporada,
será
pase
como
uno
solo
vs.
-
2:45
p.m.
4:45
o
tres
puntos
barcelona
camp
debido
lista
técnico
sí
está
problema
mediante
gustavo
gerente
deportivo
alianza
lima,
aseguró
pablo
tema
próxima
entrenador
mucho
valor
duda
nos
acuerdo
semana
ese
dijo
lima
zevallos
pero
resultados
sino
todo
jugadores
cómo
va
gran
otro
momento
versión
ello
llegar
después
veremos
qué
hacer
cada
buen
también
teniendo
garcilaso
cristal
me

In [115]:
modelo["durante"]

array([-0.10280798,  0.15010135, -0.21695147,  0.18457127,  0.02758516,
       -0.02698358,  0.01007572, -0.03469809,  0.33619612, -0.27521384], dtype=float32)

In [116]:
len(modelo.wv.vocab)

561

Creamos el vector promedio por noticia:
1. Vamos a barrer cada registro del data frame.
2. Cada noticia la spliteo para luego hacer un barrido palabra por palabra.
3. Dentro del barrido voy preguntando si la palabra esta en mi modelo, sumarizo el vector de la palabra al un vector temporal (**vector**).
4. Luego divido el vector temporal entre el total de palabras que si estaban en el modelo (**contador_palabras**), lo que vendria a ser el vector promedio de la noticia (**vector**). 

In [126]:
vector_noticias = []
for i in range(len(df)):
    lista = df['contenido'][i].split()
    contador_palabras=0
    vector = [0 for x in range(dimensiones)]
    for j in range(len(lista)):
        if lista[j] in modelo.wv.vocab:
            contador_palabras+=1
            vector+=modelo[lista[j]]
    vector = vector/contador_palabras if contador_palabras>0 else vector
    vector_noticias.append(vector)
#Cantidad de vectores
len(vector_noticias)

28

In [118]:
vector_noticias

[array([-0.53042482,  0.72809201, -1.03324797,  0.75877336,  0.02073504,
         0.08939737,  0.07367809, -0.07287226,  1.5184994 , -1.23510761]),
 array([-0.47912545,  0.65328593, -0.92884302,  0.68361388,  0.01462392,
         0.08042498,  0.07147622, -0.05661755,  1.36563386, -1.10677508]),
 array([-0.49527771,  0.67823119, -0.95973039,  0.71364105,  0.01520994,
         0.08374605,  0.0720825 , -0.0607768 ,  1.41994446, -1.15072503]),
 array([-0.45385094,  0.62295863, -0.88408032,  0.65334099,  0.012381  ,
         0.07839275,  0.06816866, -0.05555999,  1.29882777, -1.05881598]),
 array([-0.52709094,  0.71759286, -1.01728411,  0.75649892,  0.01906994,
         0.08920751,  0.07549504, -0.06719987,  1.50114369, -1.21880951]),
 array([-0.51713682,  0.71479428, -1.0117058 ,  0.75068305,  0.0151371 ,
         0.08981441,  0.07778648, -0.05837991,  1.49339561, -1.21539964]),
 array([-0.56328304,  0.76868482, -1.08796405,  0.80900589,  0.01301383,
         0.09502543,  0.07895377, -0.07

# Entrenando el modelo de Kmeans

Vamos a entrenar un modelo kmeans que agrupe las 38 noticias en 4 categorias

In [119]:
from sklearn.cluster import KMeans


In [127]:
est = KMeans(4)  # K clusters
est.fit(vector_noticias)
y_kmeans = est.predict(vector_noticias)

In [128]:
est

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [129]:
y_kmeans

array([0, 3, 3, 1, 0, 0, 2, 3, 1, 0, 3, 2, 0, 2, 0, 0, 3, 0, 2, 3, 0, 3, 0,
       3, 0, 0, 1, 2], dtype=int32)

In [130]:
est.cluster_centers_

array([[-0.88483057,  1.20123778, -1.69953745,  1.26726316],
       [-0.73563369,  1.01317893, -1.43726064,  1.07281712],
       [-0.97091284,  1.30838655, -1.8468985 ,  1.38670468],
       [-0.81836575,  1.11403317, -1.57344204,  1.17633048]])